In [2]:
import os
import numpy as np 
import pandas as pd 

import tweepy
from konlpy.tag import Okt, Mecab
from wordcloud import WordCloud, STOPWORDS
from PIL import Image
import nltk

from pprint import pprint

import warnings
warnings.filterwarnings(action='ignore')

In [3]:

current_dir = os.getcwd()
genre_df = pd.read_csv(current_dir + '/data/genre_df.csv')

In [4]:
genre_df

,genre,script
0,액션,"액션 영화(action film)는 영화의 한 갈래로, 등장인물의 육체적인 움직임을..."
1,코미디,"코미디 영화(comedy film)는 유머에 중점을 둔 영화 장르로, 관객들로부터 ..."
2,다큐멘터리,"다큐멘터리 영화(영어: Documentary Film), 또는 기록 영화(記錄映畵,..."
3,판타지,판타지 영화란 판타지적인 내용을 영화로 만든 것이다.
4,공포,"공포 영화(恐怖映畫, 영어: horror film)란 귀신이나 유령 등 무서운 것과..."
5,음악,"뮤지컬 영화(Musical film)는 영화의 장르 중 하나로, 이야기에 혼재하여 ..."
6,로맨스,"로맨스 영화(영어: romance film)는 영화의 장르 중 하나로, 사랑을 주제..."
7,스포츠,"스포츠 영화(-映畵)는 스포츠를 소재로 한 영화로, 스포츠 선수나 팀, 경기 등을 ..."
8,서부,서부극(西部劇)은 미국의 역사 중에서 서부 개척사를 소재로 한 영화나 극 작품이다.
9,Made in Europe,위키백과에 ‘Made in Europe 영화’ 문서가 존재하지 않습니다.


In [5]:
# 불용어 설정
SW = set(genre_df['genre'])
SW.add('영화')

In [6]:
def kor_preprocessing(text):
    tokenizer = Mecab()
    total_tokens = tokenizer.nouns(text)
    total_tokens = [token for token in total_tokens if len(token) > 1] # token not in SW and 
    
    return total_tokens

In [7]:
contents = list(genre_df['script'])
kor_tokens = [kor_preprocessing(doc) for doc in contents]
kor_total_tokens = []
for L in kor_tokens:
    kor_total_tokens += L
 
kor_total_tokens[:2]

['액션', '영화']

In [8]:
text = nltk.Text(kor_total_tokens)
print("토큰 개수: ", len(set(text.tokens)))
pprint(text.vocab().most_common(10))

토큰 개수:  230
[('영화', 56),
 ('장르', 20),
 ('공포', 9),
 ('소재', 7),
 ('작품', 6),
 ('역사', 6),
 ('사건', 5),
 ('전쟁', 5),
 ('포함', 5),
 ('코미디', 5)]


In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer
from collections import defaultdict
import nltk

vectorizer = TfidfVectorizer()
sp_matrix = vectorizer.fit_transform(contents)

word2id = defaultdict(lambda : 0)
for idx, feature in enumerate(vectorizer.get_feature_names()):
    word2id[feature] = idx

for i, sent in enumerate(contents):
    print('========= %s =========' % genre_df.iloc[i].genre)
    a = [ (token, sp_matrix[i, word2id[token]]) for token in kor_preprocessing(sent) if sp_matrix[i, word2id[token]]>0.1]
    a.sort(key=lambda x : x[1], reverse=True)
    b = list(set(a))
    pprint(b)

========= 액션 =========
[('재난', 0.1798251126280635),
 ('등장인물', 0.1798251126280635),
 ('권선징악', 0.1798251126280635),
 ('이야기', 0.1798251126280635),
 ('활약', 0.1798251126280635),
 ('갈래', 0.1798251126280635),
 ('해결', 0.1798251126280635),
 ('정의', 0.1798251126280635),
 ('움직임', 0.1798251126280635),
 ('주안점', 0.1798251126280635),
 ('서부극', 0.15774761676369514),
 ('포함', 0.1798251126280635),
 ('전쟁', 0.14208338416316077),
 ('육체', 0.1798251126280635),
 ('이용', 0.1798251126280635),
 ('액션', 0.28416676832632154),
 ('영화', 0.18055999190856398),
 ('의미', 0.1798251126280635),
 ('활극', 0.1798251126280635),
 ('스릴러', 0.14208338416316077)]
========= 코미디 =========
[('상황', 0.17900566676074084),
 ('행동', 0.17900566676074084),
 ('영화', 0.11982479817768756),
 ('대부분', 0.17900566676074084),
 ('방법', 0.17900566676074084),
 ('코미디', 0.4710863278107513)]
========= 다큐멘터리 =========
[('문화어', 0.20797192727144873),
 ('논픽션', 0.20797192727144873),
 ('다큐멘터리', 0.41594385454289745),
 ('기록', 0.20797192727144873),
 ('라틴어', 0.2079719272714487

In [7]:

europe_genre_keywords = ["그리스", "네덜란드", "노르웨이", "덴마크", "독일", "라트비아", "루마니아", "룩셈부르크", "리투아니아", "리히텐슈타인", "모나코", "몬테네그로", "몰도바", "몰타", "벨기에", "벨라루스", "보스니아-헤르체고비나", "북마케도니아", "불가리아", "사이프러스", '산마리노', "세르비아", "스웨덴", "스위스", "스페인", "슬로바키아", "슬로베니아", "아르메니아", "아이슬란드", "아일랜드", "아제르바이잔", "안도라", "알바니아", "에스토니아", "영국", "오스트리아", "우크라이나" , "이탈리아", "조지아", "체코", "코소보", "크로아티아", "터키", "포르투갈", "폴란드", "프랑스", "핀란드", "헝가리"]